
# DS400: Assignment 5 (Python)

##### Daniel Hinz (Student ID: 6302371)
##### January 25 2023

## Preprocessing

(1) In the folder data you will find the original images. Each image is labeled as either pizza or no pizza.
Apply the pipeline below and export to csv.

In [3]:
# Imports
import os
import numpy as np
import pandas as pd
import requests
import time



In [4]:
# set directory
os.chdir("D:/Dokumente/Studium/Master_DataScience/Assignment5")

In [5]:
# load key
with open("key.txt") as f:
  key = f.readlines()[0]

# get request to api
APIcontent = requests.get("https://app.ticketmaster.com/discovery/v2/venues",
                           params = {"apikey":key, "countryCode":"DE"}).json()

print(APIcontent)

{'_embedded': {'venues': [{'name': 'Grosse Freiheit 36', 'type': 'venue', 'id': 'KovZpZAn6vEA', 'test': False, 'url': 'http://www.ticketmaster.de/venue/287636', 'locale': 'en-de', 'images': [{'ratio': '16_9', 'url': 'https://s1.ticketm.net/dbimages/962v.', 'width': 205, 'height': 115, 'fallback': False}], 'postalCode': '22767', 'timezone': 'Europe/Berlin', 'city': {'name': 'Hamburg'}, 'country': {'name': 'Germany', 'countryCode': 'DE'}, 'address': {'line1': 'Grosse Freiheit 36'}, 'location': {'longitude': '9.957905', 'latitude': '53.551187'}, 'markets': [{'name': 'Germany', 'id': '210'}], 'dmas': [{'id': 610}], 'boxOfficeInfo': {'phoneNumberDetail': 'Telefon tagsüber: (040) 31 77 78- 0 Große Freiheit 36 22767 Hamburg E-Mail: info@grossefreiheit36.de'}, 'upcomingEvents': {'_total': 1, 'mfx-de': 1, '_filtered': 0}, 'ada': {'adaPhones': '+49.(0)1805 - 969 0000 (14 Ct./Min.)', 'adaCustomCopy': 'Sollten Sie besondere Sitzplatzwünsche haben, prüfen Sie bitte die Informationsseite des Veranst

In [6]:
print(APIcontent["page"])

{'size': 20, 'totalElements': 4745, 'totalPages': 238, 'number': 0}


In [10]:
# Function to query the nested dictionary which we receive from the API

def query_dict(dict_, *keys):
    """
    This function needs a dictionary and one or multiple keys. Output is the element stored behind the key or a None
    if that element is not present
    """
    
    # for some instances we have two keys, if that is the case, query both an get the element
    for key in keys:
        try:
            dict_ = dict_[key]
        except:
    # if that particular entry is not present return Nne
            return None
    return dict_

In [11]:
# Define output Dataframe with columns that we would like to extract
venues = pd.DataFrame(columns = ["name", "city", "postalCode", "address", "url", "longitude", "latitude"])

for i in APIcontent["_embedded"]["venues"]:
    
    objects = []
    
    objects.append(query_dict(i,"name"))
    objects.append(query_dict(i,"city","name"))
    objects.append(query_dict(i,"postalCode"))
    objects.append(query_dict(i,"address","line1"))
    objects.append(query_dict(i,"url"))
    objects.append(query_dict(i,"location","longitude"))
    objects.append(query_dict(i,"location","latitude"))

    
    venues.loc[len(venues)] = objects

In [13]:
venues.head()

,name,city,postalCode,address,url,longitude,latitude
0,Grosse Freiheit 36,Hamburg,22767,Grosse Freiheit 36,http://www.ticketmaster.de/venue/287636,9.957905,53.551187
1,Gruenspan,Hamburg,22767,Grosse Freiheit 58,http://www.ticketmaster.de/venue/287155,9.958075,53.551885
2,Huxleys Neue Welt,Berlin,10967,Hasenheide 107 – 113,http://www.ticketmaster.de/venue/286842,13.42138,52.486391
3,Virtual Event,Worldwide,None,None,https://www.ticketmaster.de/venue/virtuelles-e...,10.0,50.0
4,Ev. St. Jacobi Kirche,Sangerhausen,06526,Marktplatz,http://www.ticketmaster.de/venue/290061,None,None


### Interacting with the API advanced
We 238 pages with 20 elements each. By setting the element size to 100, we need 48 pages to query

In [14]:
# initiate output dataframe
venues_all = pd.DataFrame(columns = ["name", "city", "postalCode", "address", "url", "longitude", "latitude"])

# loop over all 48 pages
for page in range(48):
    APIcontent = requests.get("https://app.ticketmaster.com/discovery/v2/venues",
                               params = {"apikey":key, "countryCode":"DE", "page":page, "size":100}).json()
    
    # wait for 2 seconds before the next request
    time.sleep(2) 
    # some random pages produce errors. Maybe its because we are using the same public key, the query limit is reached.
    # if an error occurs, we wait some seconds and try again
    try:
       	for i in APIcontent["_embedded"]["venues"]:
               
               objects = []
               
               objects.append(query_dict(i,"name"))
               objects.append(query_dict(i,"city","name"))
               objects.append(query_dict(i,"postalCode"))
               objects.append(query_dict(i,"address","line1"))
               objects.append(query_dict(i,"url"))
               objects.append(query_dict(i,"location","longitude"))
               objects.append(query_dict(i,"location","latitude"))
               
               venues_all.loc[len(venues_all)] = objects

    except:
        time.sleep(2) 
        page = page-1
        print("error at:", page)
        
venues_all.head(n=10)

error at: -1
error at: 6


,name,city,postalCode,address,url,longitude,latitude
0,St. Gandolf,Amorbach,63916,Kirchplatz 7,http://www.ticketmaster.de/venue/290561,None,None
1,Bröltalhalle,Ruppichteroth,53809,Dr. Herzfeld Strasse,http://www.ticketmaster.de/venue/290524,None,None
2,Club Biergarten Dieburg,Dieburg,64807,Hohe Strasse 17,http://www.ticketmaster.de/venue/290778,None,None
3,Eventhaus Hoffnung,Lübeck,23564,Hüxtertorallee 23a,http://www.ticketmaster.de/venue/290771,None,None
4,Sporthalle am Böllenfalltor,Darmstadt,64285,Nieder Ramstädter Strasse 170,http://www.ticketmaster.de/venue/290765,None,None
5,Irish Harp Pup,Berlin,10629,Giesebrechtstrasse15,http://www.ticketmaster.de/venue/290767,None,None
6,Inselhalle,Lindau,88131,Ludwigstrasse 68,http://www.ticketmaster.de/venue/290782,None,None
7,4053,Madrid - Spanien,28013,Calle de Tetuan 27,http://www.ticketmaster.de/venue/290773,None,None
8,Badesee,Ertingen,88521,Schwarenachstrasse,http://www.ticketmaster.de/venue/290787,None,None
9,Kurpark Classix,Aachen,52062,Kurgarten Monheimsallee,http://www.ticketmaster.de/venue/290781,None,None


In [15]:
venues_all.shape

(4545, 7)